In [2]:
import numpy as np
import math
import random

In [3]:
class Value:
    
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._prev = set(_children)
        self._op = _op
        self.label = label
        self._backward = lambda: None
        
    def __repr__(self):
        return f"Value(label={self.label}, data={self.data}, grad={self.grad})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad  # d(other)/d(loss) = d(other)/d(out) * d(out)/d(loss) = 1.0 * out.grad
            other.grad += 1.0 * out.grad  # += to be able to do a+a, Otherwise the gradients will overwrite each other. they should be added.
        out._backward = _backward
        return out
    
    def __radd__(self, other):
        return self + other

    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        return self + (-other)

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def __rmul__(self, other):  # When it cant do mul, try the other way around. 
        return self * other     # input is other, self. Already swithced
    
    # Implementing tanh x = (e^2x - 1) / (e^2x + 1)
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self,), 'tahn')

        def _backward():
            self.grad += (1 - t**2) * out.grad  # local derivive (tanh) is: 1 - tanh(x)**2 = 1-t**2
        out._backward = _backward
        return out

    def __truediv__(self, other):
        """other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data / other.data, (self, other), '/')  # value, prev, op (no label)

        def _backward():
            self.grad += out.grad * 1.0 / other.data
            other.grad += out.grad * -1.0 * self.data / other.data**2
        out._backward = _backward"""
        return self * (other ** -1)

    def __rdiv__(self, other):
        return self / other

    def __pow__(self, other):
        # other = other if isinstance(other, Value) else Value(other)
        assert isinstance(other, (int, float)), "Only power of int or float is supported."
        out = Value(self.data ** other, (self,), f'**{other}')

        def _backward():
            self.grad += other * self.data ** (other -1) * out.grad
            #other.grad += self.data ** other * math.log(self.data) * out.grad
        out._backward = _backward
        return out

    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')

        def _backward():
            self.grad += out.data * out.grad

        out._backward = _backward
        return out
     
    def backward(self):
        # topological sort
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0  # set starting to be the last node, having gradient 1 (how much does last node change if you change itself?)
        for node in reversed(topo):
            node._backward()
    


In [4]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'

In [5]:
o.backward()
print(x1w1.grad)
print(x2w2.grad)

0.4999999999999999
0.4999999999999999


In [21]:
class Neuron:

    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        # x * w + b
        assert len(x)==len(self.w), "input length does not match"
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b) # second parameter in sum sets the start to sum from.
        out = act.tanh()                                        # obs zip cuts the logest list if not equal
        return out

    def parameters(self):
        return self.w + [self.b]

class Layer:
    
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __call__(self, x): 
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs)==1 else outs

    def parameters(self):
        #print([n.parameters() for n in self.neurons])  # this one keeps the nested lists
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:

    def __init__(self, nin, nouts):  # Nouts being a list of all layers size
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        # [l.parameters() for l in self.layers]  # this one keeps the nested lists
        return [p for layer in self.layers for p in layer.parameters()]


In [22]:
nn = MLP(3, [4, 4, 1])
xs = [
    [1.0, 2.0, 3.0], 
    [2.0, 3.0, 3.0], 
    [1.0, 1.0, 0.0], 
    [1.0, 1.0, 1.0]
]
ys = [1.0, 1.0, -1.0, -1.0]  # small values predicts -1


In [23]:
nr_epochs = 100
lr = 0.01#1e-3

for epoch in range(nr_epochs):
    # Forward pass
    ypred = [nn(x) for x in xs]
    loss = sum((yp-yn)**2 for yn, yp in zip(ys, ypred))

    # Backward pass AND zero gradients
    for p in nn.parameters():
        p.grad = 0.0
    loss.backward()

    # Uppdate weights
    
    for p in nn.parameters():
        p.data += p.grad * -lr
    if epoch % 10 == 0:
        print(loss.data)

print('-------------')
print('Predictions:')
print([y.data for y in ypred])

4.995237104644893
2.174614860351153
1.0783247466216819
0.5911642257789561
0.3648335699323164
0.24817241568023835
0.18156480411076092
0.14012151270796994
0.11251249062112018
0.09310505847357048
-------------
Predictions:
[0.9166775765494444, 0.8308331026970641, -0.9367875978554346, -0.7986015389846126]


 *Most common neural net mistakes:* 
 1) you didn't try to overfit a single batch first. 
 2) you forgot to toggle train/eval mode for the net. 
 3) you forgot to .zero_grad() (in pytorch) before .backward(). 
 4) you passed softmaxed outputs to a loss that expects raw logits.

In [35]:

for i, p in enumerate(nn.layers[0].parameters()):
    if i % 4 == 0: # 3inputs + bias
        print('')
    print(p)
print('')
print('')
nod = 3
w1s = nn.layers[0].parameters()
print(w1s[nod])
print(w1s[nod+4])
print(w1s[nod+8])
print(w1s[nod+12])


Value(label=, data=0.30747227717092224, grad=0.0035449865604668465)
Value(label=, data=-0.025424505627471106, grad=0.001155635968813692)
Value(label=, data=-0.41283872547346234, grad=0.0007267872569475826)
Value(label=, data=-1.0479397968959554, grad=0.005805603935577617)

Value(label=, data=0.4764488535167226, grad=-0.05188865748219201)
Value(label=, data=-0.3527680234272192, grad=0.03637976928107434)
Value(label=, data=-0.5009204153721958, grad=0.05111771577987495)
Value(label=, data=0.6624933808826854, grad=-0.1309237514164218)

Value(label=, data=-0.21067607403296473, grad=-0.02075844061724675)
Value(label=, data=-0.8442942844375141, grad=-0.05381450134491602)
Value(label=, data=0.8740996830269342, grad=-0.06030335399954203)
Value(label=, data=-0.6742363469928555, grad=0.006055522562564565)

Value(label=, data=-0.678639563388376, grad=-0.030987361163647247)
Value(label=, data=0.5306312174296226, grad=-0.014836785870856024)
Value(label=, data=-0.557436589917409, grad=-0.00913669488

In [16]:
nn.layers[0]